In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
cd 'drive/MyDrive/CS224N Project'

/content/drive/.shortcut-targets-by-id/1XCSZcGmhAAbXody1GLo0I2yPtdg-TBX3/CS224N Project


In [ ]:
mv recommendations.txt 'drive/CS224 Project'

In [26]:
ls

'Project Proposal.gdoc'                        reviews.txt
'[Pt1] Download Reviews (02132021T).ipynb'     top_links.csv
'[Pt1] Scrape anime (02132021M).ipynb'         top_links.gsheet
'[Pt1] Scraping Top Links (02052021R).ipynb'   user_ratings.txt
 recommendations.txt


In [2]:
import urllib
import bs4

In [3]:
def link_to_soup(link,html=False):
    source = urllib.request.urlopen(link).read()
    soup=bs4.BeautifulSoup(source,'lxml' if html else 'html.parser')
    return soup

In [4]:
def get_series(pg_start=0,pg_stop=0):
    links=[]
    for pg in range(pg_start,pg_stop+1):
        url="https://myanimelist.net/topanime.php?limit={}".format(50*pg)
        soup=link_to_soup(url)
        trs=soup.find_all("tr",class_="ranking-list")
        for tr in trs:
            td=tr.find_all('td')[1]
            links.append(td.a['href'])
    return links

In [5]:
a=get_series(0,4)

In [11]:
a[0]

'https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood'

In [14]:
import re
b=[]
pat='_+'
for name in a:
  title=re.sub(pat,' ',name.split('/')[-1])
  name_id=name.split('/')[-2]
  b.append([title,str(name_id)])



In [27]:
import numpy as np

np.savetxt("titles.txt",np.array(b),'%s')

In [20]:
pwd

'/content'

In [ ]:


def get_synopsis(soup):
    descr=soup.find('p',itemprop="description")
    return descr.text

def get_score(soup):
    rating=soup.find('span',itemprop="ratingValue")
    return float(rating.text)

def get_numerical_features(soup):
    # [#ranked, #popularity, #members, #favorites]
    extract=lambda x:''.join(list(filter(lambda y:y.isdigit(),list(x))))
    ranked_div=soup.find('div',attrs={"data-id":"info2"})
    ranked=ranked_div
    rank=extract(ranked.span.next_sibling)
    rank=int(rank)
    pop_div=ranked_div.parent
    divs=pop_div.find_all('div')
    fav=extract(divs[-2].span.next_sibling)
    members=extract(divs[-3].span.next_sibling)
    pop=extract(divs[-4].span.next_sibling)
    return [rank,pop,members,fav]

def get_reviews(soup):    
    #divs with e.g. id=score12912 identifies a user
    numeric_reviews=soup.find_all('div',attrs={"id":lambda x: x and "score" in x})
    #for each numeric review, 
    #every category is rating/10, [overall,story,animation,sound,character,enjoyment]
    numerics=[]
    for numeric_review in numeric_reviews:
        numeric=[]
        table=numeric_review.table
        for tr in table.find_all("tr"):
            numeric.append(int(tr.find_all('td')[-1].text))
        numerics.append(numeric)
        paragraphs=numeric_review.next_siblings
        review=""
        for p in paragraphs:
            if type(p)==bs4.element.Tag and p.name=="span": break
            elif type(p)==bs4.NavigableString: review+=p
        numerics.append(review)
    return numerics

    

def scrape_title(page):
    #feat_vec:[synopsis,score,numeric feats,
    #          reviewer's category ratings + reviewer's review for all reviewers]
    #e.g. "https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood"
    soup=link_to_soup(page)
    lis=[get_synopsis(soup),get_score(soup)]
    lis.extend(get_numerical_features(soup))
    i=1; a=-1; b=len(lis)
    while len(lis)>a:
        a=len(lis)
        lis.extend(get_reviews(link_to_soup(page+"/reviews?p={}".format(i))))
        i+=1
    return lis, len(lis)-b


In [ ]:
#get_reviews(soup)

In [ ]:
def link_to_id(link):
    a=link.split('/')
    ind=a.index('anime')
    return int(a[ind+1])

def get_recs():
    dic=[]
    soup=link_to_soup(rec_url)
    trs=soup.find_all("tr")
    for i in range(len(trs)):
        tr=trs[i]
        tds=tr.find_all('td')
        td1,td2=tds[0],tds[1]
        get_name=lambda td:td.div.a['href']
        if not td1 or not td2: continue
        n1=get_name(td1)
        n2=get_name(td2)
        dic.append((link_to_id(n1),link_to_id(n2)))
    return dic


In [ ]:
# #eda, count #reviews/page
# pg_links=get_series(0,0)
# cts=[]
# for link in pg_links:
#     try:
#         cts.append(scrape_title(link)[0])
#     except BaseException:
#         continue

In [ ]:
test="https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood/userrecs"
# def page_recs(page):
#     soup=link_to_soup(page)
#     recs=[]
#     content=soup.find('div',attrs={'id':"content"})
#     table=content.table
    
#     for div in table.find_all("div",class_="borderClass"):
#         href=div.table.tr.td.div.a["href"]
#         descr=""
#         recs.append((a["href"],descr))
#     return recs
# print(page_recs(test))

In [ ]:
def get_pg_recs(page):
    #for robustness, only returns anime with at least two mentions
    try:
        soup=link_to_soup(page+"/userrecs")
    except:
        print("skipping",page)
        return []
    content=soup.find('div',attrs={'id':"content"})
    table=content.table
    td=list(filter(lambda x:x.name=="td",table.tr.contents))[1]
    class_match=lambda x:("borderClass" in x.attrs.get("class",dict()))
    rec_lambda=lambda x:x.name=="div" and class_match(x)
    divs=list(filter(rec_lambda,td.contents))
    all_recs=[]
    for div in divs:
        td=div.table.tr.find_all("td")[1]
        lis=(list(filter(lambda x:x.name=="div",td.contents)))
        name=lis[1].a["href"]
        rec=lis[2].div.text
        recs=[rec]
        try:
            no_recs=int(lis[3].a.strong.text)+1
        except IndexError:
            all_recs.append([name,1,recs])
            continue
        other_recs=lis[4].contents[1::2]
        for rec in other_recs:
            recs.append(rec.div.text)
        all_recs.append([page.split('/')[-2],name.split('/')[-2],str(no_recs)])
        all_recs[-1].extend(recs)
    return all_recs

In [ ]:
def get_all_recs(start_pg=0,end_pg=0):
    recs=[]
    for link in get_series(start_pg,end_pg):
        recs.extend(get_pg_recs(link))
        print("link",link,"now at {} recs".format(len(recs)))
    return recs

In [ ]:
recs=get_all_recs(0,4)

link https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood now at 52 recs
link https://myanimelist.net/anime/40028/Shingeki_no_Kyojin__The_Final_Season now at 57 recs
link https://myanimelist.net/anime/9253/Steins_Gate now at 135 recs
link https://myanimelist.net/anime/38524/Shingeki_no_Kyojin_Season_3_Part_2 now at 137 recs
skipping https://myanimelist.net/anime/28977/Gintama°
link https://myanimelist.net/anime/28977/Gintama° now at 137 recs
link https://myanimelist.net/anime/11061/Hunter_x_Hunter_2011 now at 207 recs
link https://myanimelist.net/anime/820/Ginga_Eiyuu_Densetsu now at 245 recs
link https://myanimelist.net/anime/9969/Gintama now at 249 recs
link https://myanimelist.net/anime/15417/Gintama__Enchousen now at 251 recs
link https://myanimelist.net/anime/35180/3-gatsu_no_Lion_2nd_Season now at 254 recs
link https://myanimelist.net/anime/28851/Koe_no_Katachi now at 278 recs
link https://myanimelist.net/anime/34096/Gintama now at 280 recs
link https://myanimelist

In [ ]:
# # file1 = open("top_links.csv","w") 
# # file1.write("MAL Anime ID, Rec\n")

# with open('top_links.csv', 'r') as file:
#     reader = csv.reader(file)
#     for row in reader:
#         if(row[0] == 'MAL Anime ID'):
#             continue
#         pg = row[1]
#         all_recs=get_pg_recs(pg)
#         print(len(all_recs))

# # file1.close

In [ ]:

import numpy as np
res_recs=[]
for rec in recs: res_recs.extend(rec)



In [ ]:
res_recs

In [ ]:
with open('recommendations.txt','w+') as f:
    for rec in res_recs:
        f.write(str(rec))
        f.write('\n')

In [ ]:
len(recs)

2792

In [ ]:
res_recs[0]+'\n'len(res_recs)